# Day 3 - Lab 2: Refactoring & Documentation (Solution)

**Objective:** Use an LLM to refactor a complex Python function to improve its readability and maintainability, and then generate comprehensive, high-quality documentation for the project.

**Introduction:**
This solution notebook provides the complete prompts for the refactoring and documentation lab. It demonstrates how to guide an LLM to perform specific code quality improvements and generate structured documentation from multiple sources.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Code to Improve

In [4]:
bad_code = """
def process_data(data, operation):
    if operation == 'sum':
        total = 0
        for i in data:
            total += i
        return total
    elif operation == 'average':
        total = 0
        for i in data:
            total += i
        return total / len(data)
    elif operation == 'max':
        max_val = data[0]
        for i in data:
            if i > max_val:
                max_val = i
        return max_val
"""

## Step 3: The Challenges - Solutions

### Challenge 1 (Foundational): Refactoring the Code

**Explanation:**
This prompt is highly specific about the desired outcome. Instead of just saying "improve this code," we give the LLM concrete principles to follow: apply the 'Single Responsibility Principle,' use built-in functions, and add type hints. This guidance transforms a vague request into a precise engineering task, leading to a much higher-quality output.

In [9]:
refactor_prompt = f"""
You are a senior Python developer who writes clean, efficient, and maintainable code.

Please refactor the following Python code. Apply the 'Single Responsibility Principle' by breaking the main function into smaller, more focused functions. Also, use modern Python features like built-in functions and add type hints.

**Code to Refactor:**
```python
{bad_code}
```

Output only the refactored Python code.
"""

print("--- Refactoring Code ---")
refactored_code = get_completion(refactor_prompt, client, model_name, api_provider)
cleaned_code = clean_llm_output(refactored_code, language='python')
print(cleaned_code)

--- Refactoring Code ---
from typing import List, Union

def calculate_sum(data: List[Union[int, float]]) -> Union[int, float]:
    return sum(data)

def calculate_average(data: List[Union[int, float]]) -> float:
    return sum(data) / len(data) if data else 0.0

def find_max(data: List[Union[int, float]]) -> Union[int, float]:
    return max(data)

def process_data(data: List[Union[int, float]], operation: str) -> Union[int, float]:
    operations = {
        'sum': calculate_sum,
        'average': calculate_average,
        'max': find_max,
    }
    if operation in operations:
        return operations[operation](data)
    raise ValueError(f"Unsupported operation: {operation}")


### Challenge 2 (Intermediate): Generating Docstrings

**Explanation:**
This prompt builds on the previous step. We provide the newly refactored code and ask for another specific, structured output: Google-style docstrings. LLMs are exceptionally good at this type of structured text generation. They can parse the function signature to identify the arguments and return types and generate well-formatted, descriptive documentation.

In [11]:
docstring_prompt = f"""
You are a Python developer who writes excellent documentation.

Add Google-style docstrings to the following Python code. Each docstring should include a description of the function, its arguments (Args:), and what it returns (Returns:).

**Python Code:**
```python
{cleaned_code}
```

Output the complete Python code with the added docstrings.
"""

print("--- Generating Docstrings ---")
code_with_docstrings = get_completion(docstring_prompt, client, model_name, api_provider)
cleaned_code_with_docstrings = clean_llm_output(code_with_docstrings, language='python')
print(cleaned_code_with_docstrings)

--- Generating Docstrings ---
from typing import List, Union

def calculate_sum(data: List[Union[int, float]]) -> Union[int, float]:
    """Calculate the sum of a list of numbers.

    Args:
        data (List[Union[int, float]]): A list of integers and/or floats to be summed.

    Returns:
        Union[int, float]: The sum of the numbers in the list.
    """
    return sum(data)

def calculate_average(data: List[Union[int, float]]) -> float:
    """Calculate the average of a list of numbers.

    Args:
        data (List[Union[int, float]]): A list of integers and/or floats to calculate the average.

    Returns:
        float: The average of the numbers in the list. Returns 0.0 if the list is empty.
    """
    return sum(data) / len(data) if data else 0.0

def find_max(data: List[Union[int, float]]) -> Union[int, float]:
    """Find the maximum number in a list of numbers.

    Args:
        data (List[Union[int, float]]): A list of integers and/or floats to find the maximum value.

### Challenge 3 (Advanced): Generating a Project README

**Explanation:**
Using an LLM to generate docstrings and a README is a massive productivity boost. It excels at this structured writing task, freeing up the developer to focus on complex logic while still ensuring the project is well-documented and easy for others to understand. This prompt is a synthesis task. We provide the LLM with both high-level requirements (the PRD) and low-level implementation details (the API source code). The LLM's job is to merge these two sources of information into a single, comprehensive `README.md` file, complete with overviews, feature lists, and practical `curl` examples derived from the code.

In [14]:
# Load the necessary context files
prd_content = load_artifact("artifacts/day1_prd.md")
api_code = load_artifact("app/main.py")

readme_prompt = f"""
You are a technical writer creating a README.md file for a new open-source project.

Use the provided Product Requirements Document (PRD) for high-level context and the FastAPI source code for technical details.

**PRD Context:**
<prd>
{prd_content}
</prd>

**API Source Code:**
<code>
{api_code}
</code>

Generate a complete README.md file with the following sections:
- Project Title
- Overview (Summarize the project's purpose from the PRD)
- Features
- API Endpoints (List the available endpoints and provide a `curl` example for each one, including the POST request with a JSON body)
- Setup and Installation (Provide basic instructions on how to run the FastAPI app with uvicorn)
"""

print("--- Generating Project README ---")
if prd_content and api_code:
    readme_content = get_completion(readme_prompt, client, model_name, api_provider)
    cleaned_readme = clean_llm_output(readme_content, language='markdown')
    print(cleaned_readme)
    save_artifact(cleaned_readme, "README.md")
else:
    print("Skipping README generation because PRD or API code is missing.")

--- Generating Project README ---
# Employee Onboarding Management Platform

## Overview

The Employee Onboarding Management Platform is a comprehensive digital solution designed to streamline and enhance the new hire experience. It provides managers and HR teams with powerful tools to track, customize, and optimize the onboarding process. By centralizing tasks, training materials, progress tracking, and feedback mechanisms into a single, intuitive interface, this platform aims to accelerate time-to-productivity for new hires while reducing administrative overhead for HR teams and managers.

## Features

- **Interactive Task Management**: New hires can track and complete essential tasks via an interactive checklist.
- **Progress Monitoring**: HR managers can track onboarding progress and receive notifications about any delays.
- **Role-Specific Task Assignment**: Team leaders can assign tasks specific to their team's needs.
- **Self-Paced Learning System**: New hires have access to tra

## Lab Conclusion

Well done! You have used an LLM to perform two of the most valuable code quality tasks: refactoring and documentation. You've seen how AI can help transform messy code into a clean, maintainable structure and how it can generate comprehensive documentation from high-level project artifacts and source code. These skills are a massive productivity multiplier for any development team.

> **Key Takeaway:** LLMs excel at understanding and generating structured text, whether that structure is code or documentation. Providing a clear 'before' state (the bad code) and a clear goal (the refactoring principles) allows the AI to perform complex code transformation and documentation tasks efficiently.